In [40]:
!pip install -q gymnasium

In [46]:
import gymnasium as gym
import numpy as np
from collections import defaultdict
import random

# Khởi tạo môi trường Blackjack
env = gym.make('Blackjack-v1', render_mode='ansi')

def epsilon_greedy_policy(Q, state, epsilon=0.1):
    """Chọn hành động dựa trên epsilon-greedy."""
    if random.uniform(0, 1) < epsilon:
        return random.choice([0, 1])  # Chọn ngẫu nhiên giữa Stick và Hit
    else:
        return np.argmax(Q[state])  # Chọn hành động có Q-value cao nhất

def generate_episode(Q, env, epsilon=0.1):
    """Tạo một episode theo chính sách epsilon-greedy."""
    episode = []
    state, info = env.reset()  # Reset trả về (state, info)
    done = False

    while not done:
        action = epsilon_greedy_policy(Q, state, epsilon)
        next_state, reward, done, truncated, info = env.step(action)
        episode.append((state, action, reward))
        state = next_state

    return episode

def monte_carlo_control(env, num_episodes=1000000, gamma=1.0, epsilon=0.1):
    """Thuật toán Monte Carlo Control với epsilon-greedy policy."""
    returns_sum = defaultdict(float)  # Tổng giá trị trả về của từng (state, action)
    returns_count = defaultdict(float)  # Số lần gặp từng (state, action)
    Q = defaultdict(lambda: np.zeros(env.action_space.n))  # Q-table

    for i_episode in range(1, num_episodes + 1):
        if i_episode % 100000 == 0:
            print(f"Episode {i_episode}/{num_episodes}")

        episode = generate_episode(Q, env, epsilon)
        G = 0  # Giá trị trả về tích lũy (Return)
        visited_state_action_pairs = set()  # Để theo dõi các cặp (state, action) đã gặp

        # Duyệt ngược qua episode để cập nhật Q-values
        for state, action, reward in reversed(episode):
            G = gamma * G + reward  # Tính Return từ cuối episode về đầu
            if (state, action) not in visited_state_action_pairs:
                visited_state_action_pairs.add((state, action))
                # Cập nhật tổng Return và số lần gặp của (state, action)
                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1
                # Cập nhật Q-value trung bình
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]

    return Q

# Chạy Monte Carlo Control cho Blackjack
Q = monte_carlo_control(env)

# Hiển thị một số Q-values đã học
for state, actions in list(Q.items())[:10]:
    print(f"State: {state}, Actions: {actions}")


Episode 100000/1000000
Episode 200000/1000000
Episode 300000/1000000
Episode 400000/1000000
Episode 500000/1000000
Episode 600000/1000000
Episode 700000/1000000
Episode 800000/1000000
Episode 900000/1000000
Episode 1000000/1000000
State: (13, 1, 0), Actions: [-0.77512777 -0.57344667]
State: (17, 1, 0), Actions: [-0.66033463 -0.68649613]
State: (8, 10, 0), Actions: [-0.5959596 -0.3261522]
State: (14, 8, 0), Actions: [-0.48737864 -0.39336137]
State: (18, 3, 0), Actions: [ 0.15953654 -0.59633028]
State: (13, 7, 0), Actions: [-0.42352941 -0.31945107]
State: (13, 6, 0), Actions: [-0.14989045 -0.24951267]
State: (20, 10, 0), Actions: [ 0.42911711 -0.85859479]
State: (18, 10, 0), Actions: [-0.23975662 -0.67743764]
State: (12, 10, 0), Actions: [-0.59594007 -0.44078436]
